This notebook calculates similarity and error between protein embeddings and use GO semantic similarity as gold standart.

In [1]:
import pandas as pd
import numpy as np
import gzip
import itertools
import multiprocessing
import csv
import pickle
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm, tqdm_notebook
from multiprocessing import Manager, Pool
from scipy.spatial.distance import cdist
from numpy.linalg import norm
from scipy.stats import spearmanr, pearsonr
import random

## Load protein vectors of ProtVec

In [2]:
apaacVectorRaw = pd.read_csv('/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/Apaac/apaac_dim80.tsv', sep='\t')

In [3]:
apaacVectorRaw[0:5]

,Protein_Id,Protein_Name,Pc1.A,Pc1.R,Pc1.N,Pc1.D,Pc1.C,Pc1.Q,Pc1.E,Pc1.G,...,Pc2.Hydrophobicity.26,Pc2.Hydrophilicity.26,Pc2.Hydrophobicity.27,Pc2.Hydrophilicity.27,Pc2.Hydrophobicity.28,Pc2.Hydrophilicity.28,Pc2.Hydrophobicity.29,Pc2.Hydrophilicity.29,Pc2.Hydrophobicity.30,Pc2.Hydrophilicity.30
0,A0A584,TVBK2_HUMAN,11.339625,5.154375,3.092625,5.154375,4.123500,9.277875,5.154375,8.247000,...,0.006514,0.002316,0.001619,-0.001182,0.001716,-0.004127,-0.011943,-0.004408,-0.000258,0.001818
1,Q9BXU3,TX13A_HUMAN,41.827552,24.918542,5.339687,16.909010,5.339687,18.688906,38.267760,22.248698,...,0.000428,0.001691,-0.003082,0.000122,0.001192,0.002445,-0.002766,-0.002308,0.001912,0.002474
2,Q15031,SYLM_HUMAN,72.439187,41.114133,19.578159,44.050857,15.662527,47.966489,48.945397,50.903212,...,0.000132,-0.000131,0.000797,-0.000286,-0.001230,0.001217,-0.000272,-0.000962,0.001050,0.002627
3,Q6PKC3,TXD11_HUMAN,69.177489,52.613865,34.101579,34.101579,23.383940,46.767880,70.151819,46.767880,...,0.002130,0.001777,-0.000881,0.000101,0.001302,0.001622,-0.001295,-0.000381,0.000220,0.001340
4,P42681,TXK_HUMAN,26.902095,32.666830,21.137360,12.490258,14.411837,20.176571,48.039455,25.941306,...,0.000680,0.001407,-0.002114,-0.002314,0.004900,0.003805,0.003160,0.005511,-0.001850,-0.003992


In [4]:
len(apaacVectorRaw.columns)

82

In [5]:
apaacVectorRaw.iloc[0][2:82]

Pc1.A                        11.3396
Pc1.R                        5.15438
Pc1.N                        3.09263
Pc1.D                        5.15438
Pc1.C                         4.1235
Pc1.Q                        9.27788
Pc1.E                        5.15438
Pc1.G                          8.247
Pc1.H                        1.03088
Pc1.I                        6.18525
Pc1.L                         16.494
Pc1.K                        7.21613
Pc1.M                        1.03088
Pc1.F                        3.09263
Pc1.P                        5.15438
Pc1.S                        9.27788
Pc1.T                         4.1235
Pc1.W                        3.09263
Pc1.Y                         4.1235
Pc1.V                        6.18525
Pc2.Hydrophobicity.1    -3.11879e-05
Pc2.Hydrophilicity.1      0.00540022
Pc2.Hydrophobicity.2      0.00329237
Pc2.Hydrophilicity.2      0.00416807
Pc2.Hydrophobicity.3     -0.00576136
Pc2.Hydrophilicity.3      0.00183126
Pc2.Hydrophobicity.4     -0.00889854
P

In [6]:
# UNIPROT data for mapping between UNIPROT accession numbers and UNIPROT entry names
uniprot_metadata_directory = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/Uniprot/"
uniprot_metadata_file_path = uniprot_metadata_directory + "uniprot_human_all.tab"
uniprot_vars = ['Entry','Entry name','Status','Protein names','Gene names','Organism','Length','Annotation' ]
uniprot_df = pd.read_csv(uniprot_metadata_file_path, sep='\t', header=None, names=uniprot_vars)

In [7]:
apaacVectorDF = pd.DataFrame(columns=['Entry', 'Vector'])

for index,row in tqdm_notebook(apaacVectorRaw.iterrows()):
    entry = row['Protein_Id']
    vector = list(apaacVectorRaw.iloc[index][2:82])
    apaacVectorDF.loc[index] = [entry,vector]


In [8]:
apaacVectorDF[0:5]

,Entry,Vector
0,A0A584,"[11.339625151941819, 5.154375069064463, 3.0926..."
1,Q9BXU3,"[41.82755201294838, 24.918541624735198, 5.3396..."
2,Q15031,"[72.43918692811772, 41.114133121364105, 19.578..."
3,Q6PKC3,"[69.17748855836737, 52.613864537349826, 34.101..."
4,P42681,"[26.90209490248905, 32.66682952445098, 21.1373..."


In [9]:
apaacVectorDF.to_pickle("/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/embedding_dataframes/apaac_dataframe.pkl")

In [13]:
protein_names = apaacVectorDF['Entry'].tolist()

In [14]:
# define similarity_list and proteinList as global variables
proteinList = []
manager = Manager()
similarity_list = manager.list()
proteinListNew = manager.list()

def parallelSimilarity(paramList):
    i = paramList[0]
    j = paramList[1] 
    aspect = paramList[2]
    if j>i:
        protein1 = proteinListNew[i]
        protein2 = proteinListNew[j]
        if protein1 in protein_names and protein2 in protein_names:
            prot1vec = np.asarray(apaacVectorDF.query("Entry == @protein1")['Vector'].tolist())
            prot2vec = np.asarray(apaacVectorDF.query("Entry == @protein2")['Vector'].tolist())
            #cosine will return in shape of input vectors first dimension
            #print(str(prot1name) + str(prot1vec))
            #print(str(prot2name) + str(prot2vec))
            cos = cosine(prot1vec.reshape(1,-1),prot2vec.reshape(1,-1)).item()
            manhattanDist = cdist(prot1vec.reshape(1,-1), prot2vec.reshape(1,-1), 'cityblock')
            manhattanDistNorm = manhattanDist/(norm(prot1vec,1) + norm(prot2vec,1))
            euclideanDist = cdist(prot1vec.reshape(1,-1), prot2vec.reshape(1,-1), 'euclidean')
            euclideanDistNorm = euclideanDist/(norm(prot1vec,2) + norm(prot2vec,2)) 
            real = paramList[3]
            #real = human_protienSimilarityMatrix.loc[protein1,protein2]
            # To ensure real and calculated values appended to same postion they saved similtanously and then decoupled
            similarity_list.append((real, cos,1-manhattanDistNorm.item(),1-euclideanDistNorm.item()))

    return similarity_list


## Calculate similarity values with parallel processing

In [15]:
def calculateMSEforOntology(aspect,matrix_type):
    
    #Clear lists before each aspect
    similarity_list[:] = []
    proteinListNew[:] = []

    similarityMatrixNameDict = {}
    similarityMatrixNameDict["All"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix.csv" 
    similarityMatrixNameDict["500"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv"
    similarityMatrixNameDict["Sparse"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv" 
    similarityMatrixNameDict["200"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix_for_highest_annotated_200_proteins.csv"
            
    similarityMatrixFileName = similarityMatrixNameDict[matrix_type]
        
    human_proteinSimilarityMatrix = pd.read_csv(similarityMatrixFileName)
    human_proteinSimilarityMatrix.set_index(human_proteinSimilarityMatrix.columns, inplace = True)
    #proteinList = human_proteinSimilarityMatrix
    proteinList = human_proteinSimilarityMatrix.columns
    
     #proteinListNew is referanced using Manager
    for prot in proteinList:
        proteinListNew.append(prot)
    if matrix_type == "Sparse":
        #sparsified_similarities = np.load("SparsifiedSimilarites_for_highest_500.npy")
        sparsified_similarity_coordinates = \
        np.load("./auxilary_input/SparsifiedSimilarityCoordinates_"+aspect+"_for_highest_500.npy")
        protParamList = sparsified_similarity_coordinates
    else:     
        i = range(len(proteinList))
        j = range(len(proteinList))
        protParamList = list(itertools.product(i,j))
    protParamListNew = []
    # Prepare parameters for parallel processing these parameters will be 
    # used concurrently by different processes
    for tup in tqdm_notebook(protParamList):
        i = tup[0]
        j = tup[1]
        
        if matrix_type == "Sparse":
            protein1 = proteinListNew[i]
            protein2 = proteinListNew[j]
            real = human_proteinSimilarityMatrix.loc[protein1,protein2]
            tupNew = (tup[0],tup[1],aspect,real)
            protParamListNew.append(tupNew)
        else:
            if j > i:
                protein1 = proteinListNew[i]
                protein2 = proteinListNew[j]
                real = human_proteinSimilarityMatrix.loc[protein1,protein2]
                tupNew = (tup[0],tup[1],aspect,real)
                protParamListNew.append(tupNew)

    total_task_num=len(protParamListNew)
    pool = Pool()
    similarity_listRet = []
    for similarity_listRet in tqdm_notebook(pool.imap_unordered(parallelSimilarity, protParamListNew), total=total_task_num):
        pass
    pool.close()
    pool.join()

    real_distance_list = [value[0] for value in similarity_listRet]
    cosine_distance_list = [value[1] for value in similarity_listRet]
    manhattan_distance_list = [value[2] for value in similarity_listRet]
    euclidian_distance_list = [value[3] for value in similarity_listRet]

    #mseValue = mse(real_distance_list,cosine_distance_list)
    cosineCorr = spearmanr(real_distance_list, cosine_distance_list)
    manhattanCorr = spearmanr(real_distance_list, manhattan_distance_list)
    euclidianCorr = spearmanr(real_distance_list, euclidian_distance_list)   
    
    print("Cosine Correlation for "+aspect+" is " + str(cosineCorr))
    print("Manhattan Correlation for "+aspect+" is " + str(manhattanCorr))
    print("Euclidian Correlation for "+aspect+" is " + str(euclidianCorr))

    return (cosineCorr,manhattanCorr,euclidianCorr)
    

In [16]:
buffer = "aspect,cosineCorr,cosineCorrPVal,manhattanCorr,manhattanCorrPVal,euclidianCorr,euclidianCorrPVal \n"

for similarity_matrix_type in ["Sparse","200","500","All"]:
    saveFileName = "SimilarityApaac_"+similarity_matrix_type+".csv"
    f = open(saveFileName,'w')
    f.write(buffer)
    for aspect in ["MF","BP","CC"]:
        corr = calculateMSEforOntology(aspect,similarity_matrix_type) 
        buffer = "" + aspect + ","+ str(corr[0][0])+ ","+ str(corr[0][1])\
        + ","+ str(corr[1][0])+ ","+ str(corr[1][1])+ ","+ str(corr[2][0])+ ","+ str(corr[2][1])+"\n" 
        f = open(saveFileName,'a')
        f.write(buffer) #Give your csv text here.
        ## Python will convert \n to os.linesep
        f.close()
    
# 0.3673674654105104 mse for MF with 0:10
# 0.31965355246378196 mse for BP with 0:10
# 0.29460915219361683 mse for CC with 0:10


Cosine Correlation for MF is SpearmanrResult(correlation=-0.003388395942953067, pvalue=0.9578319925189998)
Manhattan Correlation for MF is SpearmanrResult(correlation=0.21191641661108007, pvalue=0.0008230590682670462)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.16765164049227865, pvalue=0.00841880859270916)



Cosine Correlation for BP is SpearmanrResult(correlation=0.21991912390065518, pvalue=0.0004986374345098746)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.24322794063777123, pvalue=0.00011274652475212616)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.27613820607319295, pvalue=1.0638855689866112e-05)



Cosine Correlation for CC is SpearmanrResult(correlation=0.07549714807954372, pvalue=0.23905385922775046)
Manhattan Correlation for CC is SpearmanrResult(correlation=0.2815020930403349, pvalue=7.659529336041172e-06)
Euclidian Correlation for CC is SpearmanrResult(correlation=0.22822707651633226, pvalue=0.0003161428767538099)



Cosine Correlation for MF is SpearmanrResult(correlation=0.043015323877798974, pvalue=1.275232485371586e-09)
Manhattan Correlation for MF is SpearmanrResult(correlation=0.09287451352473554, pvalue=2.2423839488099827e-39)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.07315745666738645, pvalue=4.980173593830242e-25)



Cosine Correlation for BP is SpearmanrResult(correlation=0.03596742242916577, pvalue=3.8718513219940197e-07)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.0676074547657732, pvalue=1.3282449969712167e-21)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.06990830039941925, pvalue=5.437903528423845e-23)


...


Cosine Correlation for BP is SpearmanrResult(correlation=0.01855687245858002, pvalue=7.249096711297943e-11)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.0453083211252954, pvalue=5.004152815275431e-57)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.04563678936500136, pvalue=7.853138555122174e-58)



Cosine Correlation for CC is SpearmanrResult(correlation=0.04768344174059898, pvalue=1.7568688338370676e-62)
Manhattan Correlation for CC is SpearmanrResult(correlation=0.0393651568029345, pvalue=3.8739505895433425e-43)
Euclidian Correlation for CC is SpearmanrResult(correlation=0.04689006995592237, pvalue=1.7720138674224546e-60)



Cosine Correlation for MF is SpearmanrResult(correlation=0.04719852506253455, pvalue=0.0)
Manhattan Correlation for MF is SpearmanrResult(correlation=-0.0128362444544698, pvalue=1.3159712381639728e-171)
Euclidian Correlation for MF is SpearmanrResult(correlation=-0.009299848411743023, pvalue=5.175585260637845e-91)



Cosine Correlation for BP is SpearmanrResult(correlation=0.02514829518726819, pvalue=0.0)
Manhattan Correlation for BP is SpearmanrResult(correlation=-0.001790633016345364, pvalue=6.762570160673659e-15)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.008741840160696235, pvalue=0.0)



Cosine Correlation for CC is SpearmanrResult(correlation=0.04973499926885012, pvalue=0.0)
Manhattan Correlation for CC is SpearmanrResult(correlation=0.021906435037225103, pvalue=0.0)
Euclidian Correlation for CC is SpearmanrResult(correlation=0.03069548769339628, pvalue=0.0)
